In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab
from datetime import datetime
from pandas.plotting import scatter_matrix
import seaborn as sns
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

%matplotlib inline
sns.set_style("whitegrid")

# Import Data

In [2]:
claimJ12 = pd.read_csv('claimData-12-J10.xls')
claimJ12.head()

,CLAIM,CLAIMDATE,CUSTOMER,OCCURRENCE,CLAIM_STATUS,HOSPITALID,AREA,CLAIMAMOUNT,CAUSE,PRODUCT_CD,ANNUAL_PAYMENT_AMT,ISSUE_DT,GENDER_TYPE,AGE
0,35392034,27-NOV-15,64570415,1,12,84571001,1,18.00,J12,P10017,2000.0,30-SEP-15,2,29
1,34692346,10-MAY-16,60870157,1,10,84571003,1,17854.64,J12,P10017,2188.0,08-SEP-13,1,24
2,34525166,10-MAR-16,63171535,1,10,84571003,1,4325.80,J12,P10017,2188.0,17-AUG-14,1,30
3,34382946,07-JAN-16,64597346,1,10,84571003,1,NaN,J12,24261B,81238.4,14-OCT-15,1,42
4,32971670,30-MAY-14,60272293,1,10,84571003,1,2953.98,J12,P10017,2188.0,09-MAY-13,2,40


In [3]:
claimJ12.describe()

,CLAIM,CUSTOMER,OCCURRENCE,CLAIM_STATUS,HOSPITALID,AREA,CLAIMAMOUNT,ANNUAL_PAYMENT_AMT,GENDER_TYPE,AGE
count,8.575600e+04,8.575600e+04,85756.000000,85756.000000,8.575600e+04,85756.000000,6.468600e+04,8.575600e+04,85756.000000,85756.000000
mean,2.818454e+07,5.650525e+07,1.205035,9.877035,8.457449e+07,3.044079,1.567435e+04,1.449664e+04,1.465985,47.685701
std,3.997075e+06,4.194084e+06,0.712204,0.930369,3.997166e+03,2.204023,6.122950e+04,4.627249e+04,0.498845,13.301568
min,2.118066e+07,5.028543e+07,1.000000,1.000000,8.457100e+07,1.000000,0.000000e+00,0.000000e+00,1.000000,1.000000
25%,2.481699e+07,5.264189e+07,1.000000,10.000000,8.457106e+07,1.000000,2.280000e+03,1.650000e+03,1.000000,40.000000
50%,2.828792e+07,5.578397e+07,1.000000,10.000000,8.457260e+07,2.000000,4.792000e+03,3.945600e+03,1.000000,49.000000
75%,3.164989e+07,6.092526e+07,1.000000,10.000000,8.457580e+07,5.000000,1.192800e+04,1.474470e+04,2.000000,57.000000
max,3.543036e+07,6.536790e+07,31.000000,14.000000,8.458334e+07,10.000000,2.724834e+06,2.490800e+06,2.000000,117.000000


In [4]:
claimJ12.columns

Index(['CLAIM', 'CLAIMDATE', 'CUSTOMER', 'OCCURRENCE', 'CLAIM_STATUS',
       'HOSPITALID', 'AREA', 'CLAIMAMOUNT', 'CAUSE', 'PRODUCT_CD',
       'ANNUAL_PAYMENT_AMT', 'ISSUE_DT', 'GENDER_TYPE', 'AGE'],
      dtype='object')

In [5]:
claimJ12.groupby('GENDER_TYPE').size()

GENDER_TYPE
1    45795
2    39961
dtype: int64

In [6]:
claimJ12.groupby('OCCURRENCE').size()

OCCURRENCE
1     73823
2      8996
3      1822
4       524
5       248
6       126
7        86
8        67
9        26
10       12
11       10
12        6
13        1
31        9
dtype: int64

In [7]:
claimJ12.groupby('AGE').size()

AGE
1         1
2         5
3        40
4        50
5       135
6        89
7       163
8       117
9       206
10      243
11      132
12      216
13      236
14      162
15      199
16      206
17      206
18      219
19      244
20      325
21      511
22      439
23      422
24      476
25      548
26      711
27      719
28      837
29      776
30      786
       ... 
53     2535
54     2925
55     2816
56     2617
57     2667
58     2485
59     2266
60     2199
61     1987
62     2147
63     1657
64     1490
65     1314
66     1233
67      869
68      827
69      715
70      487
71      410
72      274
73      224
74      208
75      130
76       60
77       24
78       19
79        2
80        8
81        1
117       1
Length: 82, dtype: int64

In [8]:
claimJ12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85756 entries, 0 to 85755
Data columns (total 14 columns):
CLAIM                 85756 non-null int64
CLAIMDATE             85756 non-null object
CUSTOMER              85756 non-null int64
OCCURRENCE            85756 non-null int64
CLAIM_STATUS          85756 non-null int64
HOSPITALID            85756 non-null int64
AREA                  85756 non-null int64
CLAIMAMOUNT           64686 non-null float64
CAUSE                 85756 non-null object
PRODUCT_CD            85756 non-null object
ANNUAL_PAYMENT_AMT    85756 non-null float64
ISSUE_DT              85756 non-null object
GENDER_TYPE           85756 non-null int64
AGE                   85756 non-null int64
dtypes: float64(2), int64(8), object(4)
memory usage: 7.9+ MB


### Convert quality
GENDER_TYPE --> GenderB (M=1, F=2)  
ANNUAL_PAYMENT_AMT --> int64  
CLAIMAMOUNT --> int64    
ANNUAL_PAYMENT_AMT --> int64    

### Age Range  

# Data wrangling

In [9]:
claimJ12 = claimJ12[claimJ12.CLAIMAMOUNT >= 0]
claimJ12 = claimJ12[claimJ12.ANNUAL_PAYMENT_AMT >= 0]

In [10]:
# Convert types
claimJ12.ANNUAL_PAYMENT_AMT  =  claimJ12.ANNUAL_PAYMENT_AMT.apply(np.int64)
claimJ12.CLAIMAMOUNT   =  claimJ12.CLAIMAMOUNT .apply(np.int64)
claimJ12['GENDER_TYPEB'] = claimJ12['GENDER_TYPE'].apply(lambda x: 0 if x == 2 else 1) # map 2(F) to 0
claimJ12.describe()

,CLAIM,CUSTOMER,OCCURRENCE,CLAIM_STATUS,HOSPITALID,AREA,CLAIMAMOUNT,ANNUAL_PAYMENT_AMT,GENDER_TYPE,AGE,GENDER_TYPEB
count,6.468600e+04,6.468600e+04,64686.000000,64686.000000,6.468600e+04,64686.000000,6.468600e+04,6.468600e+04,64686.000000,64686.000000,64686.000000
mean,2.831990e+07,5.686052e+07,1.200383,9.885694,8.457442e+07,2.992719,1.567429e+04,1.361007e+04,1.455384,46.859398,0.544616
std,4.059610e+06,4.235685e+06,0.673929,0.926509,3.959840e+03,2.173105,6.122949e+04,4.668492e+04,0.498009,14.040192,0.498009
min,2.118066e+07,5.028543e+07,1.000000,1.000000,8.457100e+07,1.000000,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000
25%,2.502069e+07,5.288150e+07,1.000000,10.000000,8.457106e+07,1.000000,2.280000e+03,1.688000e+03,1.000000,38.000000,0.000000
50%,2.855813e+07,5.632359e+07,1.000000,10.000000,8.457259e+07,2.000000,4.792000e+03,3.290000e+03,1.000000,49.000000,1.000000
75%,3.182682e+07,6.113838e+07,1.000000,10.000000,8.457562e+07,4.000000,1.192800e+04,1.340000e+04,2.000000,57.000000,1.000000
max,3.543036e+07,6.536790e+07,13.000000,14.000000,8.458334e+07,10.000000,2.724833e+06,2.490800e+06,2.000000,117.000000,1.000000


In [11]:
def ageRange (x):
    if x < 0 : return 0               
    elif x >= 0 and x < 10 : return 1  
    elif x >=10 and x < 20 : return 2  
    elif x >=20 and x < 30 : return 3 
    elif x >=30 and x < 40 : return 4 
    elif x >=40 and x < 50 : return 5 
    elif x >=50 and x < 60 : return 6  
    elif x >=60 and x < 70 : return 7 
    elif x >=70 and x < 80 : return 8 
    elif x >=80 and x < 90 : return 9  
    else: return 10                    

claimJ12['AGE_CLASS']= claimJ12['AGE'].apply(ageRange)

In [12]:
claimJ12.describe()

,CLAIM,CUSTOMER,OCCURRENCE,CLAIM_STATUS,HOSPITALID,AREA,CLAIMAMOUNT,ANNUAL_PAYMENT_AMT,GENDER_TYPE,AGE,GENDER_TYPEB,AGE_CLASS
count,6.468600e+04,6.468600e+04,64686.000000,64686.000000,6.468600e+04,64686.000000,6.468600e+04,6.468600e+04,64686.000000,64686.000000,64686.000000,64686.000000
mean,2.831990e+07,5.686052e+07,1.200383,9.885694,8.457442e+07,2.992719,1.567429e+04,1.361007e+04,1.455384,46.859398,0.544616,5.234394
std,4.059610e+06,4.235685e+06,0.673929,0.926509,3.959840e+03,2.173105,6.122949e+04,4.668492e+04,0.498009,14.040192,0.498009,1.432872
min,2.118066e+07,5.028543e+07,1.000000,1.000000,8.457100e+07,1.000000,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000,1.000000
25%,2.502069e+07,5.288150e+07,1.000000,10.000000,8.457106e+07,1.000000,2.280000e+03,1.688000e+03,1.000000,38.000000,0.000000,4.000000
50%,2.855813e+07,5.632359e+07,1.000000,10.000000,8.457259e+07,2.000000,4.792000e+03,3.290000e+03,1.000000,49.000000,1.000000,5.000000
75%,3.182682e+07,6.113838e+07,1.000000,10.000000,8.457562e+07,4.000000,1.192800e+04,1.340000e+04,2.000000,57.000000,1.000000,6.000000
max,3.543036e+07,6.536790e+07,13.000000,14.000000,8.458334e+07,10.000000,2.724833e+06,2.490800e+06,2.000000,117.000000,1.000000,10.000000


In [13]:
claimJ12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64686 entries, 0 to 85755
Data columns (total 16 columns):
CLAIM                 64686 non-null int64
CLAIMDATE             64686 non-null object
CUSTOMER              64686 non-null int64
OCCURRENCE            64686 non-null int64
CLAIM_STATUS          64686 non-null int64
HOSPITALID            64686 non-null int64
AREA                  64686 non-null int64
CLAIMAMOUNT           64686 non-null int64
CAUSE                 64686 non-null object
PRODUCT_CD            64686 non-null object
ANNUAL_PAYMENT_AMT    64686 non-null int64
ISSUE_DT              64686 non-null object
GENDER_TYPE           64686 non-null int64
AGE                   64686 non-null int64
GENDER_TYPEB          64686 non-null int64
AGE_CLASS             64686 non-null int64
dtypes: int64(12), object(4)
memory usage: 7.4+ MB


In [14]:
claimJ12.head()

,CLAIM,CLAIMDATE,CUSTOMER,OCCURRENCE,CLAIM_STATUS,HOSPITALID,AREA,CLAIMAMOUNT,CAUSE,PRODUCT_CD,ANNUAL_PAYMENT_AMT,ISSUE_DT,GENDER_TYPE,AGE,GENDER_TYPEB,AGE_CLASS
0,35392034,27-NOV-15,64570415,1,12,84571001,1,18,J12,P10017,2000,30-SEP-15,2,29,0,3
1,34692346,10-MAY-16,60870157,1,10,84571003,1,17854,J12,P10017,2188,08-SEP-13,1,24,1,3
2,34525166,10-MAR-16,63171535,1,10,84571003,1,4325,J12,P10017,2188,17-AUG-14,1,30,1,4
4,32971670,30-MAY-14,60272293,1,10,84571003,1,2953,J12,P10017,2188,09-MAY-13,2,40,0,5
8,33228721,17-MAR-16,62247348,1,10,84571003,1,3349,J12,P10016,2700,26-NOV-98,1,46,1,5


In [15]:
dcf = claimJ12.groupby(['HOSPITALID'],as_index=False).count()

In [16]:
claimJ12_sort = dcf.sort_values(['CLAIM'],ascending=False).head(100)
claimJ12_re = claimJ12_sort['HOSPITALID']

In [17]:
claimJ12_data = claimJ12.loc[claimJ12['HOSPITALID'].isin(claimJ12_re)]
claimJ12_data.describe()

,CLAIM,CUSTOMER,OCCURRENCE,CLAIM_STATUS,HOSPITALID,AREA,CLAIMAMOUNT,ANNUAL_PAYMENT_AMT,GENDER_TYPE,AGE,GENDER_TYPEB,AGE_CLASS
count,4.786000e+04,4.786000e+04,47860.000000,47860.000000,4.786000e+04,47860.000000,4.786000e+04,4.786000e+04,47860.000000,47860.000000,47860.000000,47860.000000
mean,2.832289e+07,5.682145e+07,1.234622,9.875407,8.457371e+07,2.675198,1.750721e+04,1.389522e+04,1.476285,46.934246,0.523715,5.242666
std,4.069230e+06,4.248713e+06,0.739617,0.953417,3.624837e+03,2.140385,6.859981e+04,4.797642e+04,0.499442,14.233649,0.499442,1.450131
min,2.118571e+07,5.028543e+07,1.000000,1.000000,8.457101e+07,1.000000,0.000000e+00,0.000000e+00,1.000000,2.000000,0.000000,1.000000
25%,2.500239e+07,5.282743e+07,1.000000,10.000000,8.457105e+07,1.000000,2.515750e+03,1.688000e+03,1.000000,38.000000,0.000000,4.000000
50%,2.855531e+07,5.622955e+07,1.000000,10.000000,8.457162e+07,1.000000,5.192000e+03,3.288000e+03,1.000000,49.000000,1.000000,5.000000
75%,3.184581e+07,6.113396e+07,1.000000,10.000000,8.457464e+07,4.000000,1.302000e+04,1.350125e+04,2.000000,57.000000,1.000000,6.000000
max,3.543036e+07,6.534028e+07,13.000000,14.000000,8.458329e+07,7.000000,2.724833e+06,2.490800e+06,2.000000,117.000000,1.000000,10.000000


In [18]:
# Select support feature
feature_list = [  
                  'AGE_CLASS'
                , 'ANNUAL_PAYMENT_AMT'
                , 'GENDER_TYPEB'
                ]

In [19]:
X = claimJ12_data.loc[:, feature_list]
X

,AGE_CLASS,ANNUAL_PAYMENT_AMT,GENDER_TYPEB
66,6,2550,1
67,6,20287,1
68,7,32113,0
69,4,1680,1
70,6,25000,0
71,7,5600,0
72,6,1500,0
73,4,1670,1
74,4,7208,0
75,3,7286,0


In [20]:
# Train model to get 1 most nearest neigbor
nnb = NearestNeighbors(n_neighbors=10)

In [21]:
nbrs = nnb.fit(X)

In [22]:
t = [6, 2550, 1]

In [23]:
# Calculate most nearest to t[]
print(nbrs.kneighbors([t]))

(array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), array([[ 3727, 20906, 22250,  3409, 29146, 15146, 12805,  5358, 46184,
        41261]], dtype=int32))


In [24]:
claimJ12_data.iloc[3727]

CLAIM                  34429493
CLAIMDATE             18-JAN-16
CUSTOMER               64855237
OCCURRENCE                    1
CLAIM_STATUS                 10
HOSPITALID             84574110
AREA                          4
CLAIMAMOUNT               14136
CAUSE                       J12
PRODUCT_CD               P10016
ANNUAL_PAYMENT_AMT         2550
ISSUE_DT              16-JAN-16
GENDER_TYPE                   1
AGE                          57
GENDER_TYPEB                  1
AGE_CLASS                     6
Name: 5903, dtype: object

### Save model to file using pickle

In [26]:
import pickle

In [28]:
with open('hospital_recom.pickle','wb') as f:
    pickle.dump(nnb,f)

### Load model and test

In [29]:
with open('hospital_recom.pickle','rb') as f:
    loadhospital_recom = pickle.load(f)

In [30]:
# Test recommender
print(loadhospital_recom.kneighbors([t]))

(array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), array([[ 3727, 20906, 22250,  3409, 29146, 15146, 12805,  5358, 46184,
        41261]], dtype=int32))
